# Explicación general del frontend en Streamlit

El script de BioGest construye una aplicación web en Streamlit para clasificar gestos de mano a partir de una señal EMG (electromiografía). 
El flujo principal que debe seguri el usuario es:

1. Correr el código para tener la app.
2. Cargar un archivo CSV con una ventana de señal EMG ya preprocesada (por ejemplo, RMS o magnitud combinada).
3. Visualizar la señal cargada.
4. Enviar esa señal al modelo de Edge Impulse para clasificar el gesto (extensión, grip o desviación ulnar).
5. Ver la clase predicha y las probabilidades de cada gesto.

Como toda app de Streamlit, el script se ejecuta de arriba hacia abajo cada vez que el usuario interactúa (por ejemplo, cuando sube un archivo o presiona un botón). Las funciones y variables globales (como el `runner` del modelo) se usan para mantener el comportamiento consistente durante las interacciones.

## 1. Importación de librerías y contexto general

En la primera parte se importan las librerías necesarias:

- `streamlit`: para construir la interfaz web.
- `pandas`: para leer y manipular el CSV con la señal EMG.
- `numpy`: para manejar arreglos numéricos.
- `matplotlib.pyplot`: para graficar la señal.
- `ImpulseRunner` de `edge_impulse_linux.runner`: para cargar y ejecutar el modelo `.eim` exportado desde Edge Impulse.

Estas librerías permiten conectar el frontend (interfaz) con el backend de inferencia (modelo de IA).


In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from edge_impulse_linux.runner import ImpulseRunner


## 2. Configuración de la página de Streamlit

Aquí se definen parámetros básicos de la aplicación:

- `page_title`: nombre que aparece en la pestaña del navegador.
- `page_icon`: ícono de la pestaña (un emoji en este caso).
- `layout`: cómo se distribuye el ancho de la página (`centered` la deja centrada).

Esto afecta solo la presentación general, no la lógica de clasificación.


In [ ]:
st.set_page_config(
    page_title="BioGest EMG – Clasificación de Gestos",
    page_icon="🧠",
    layout="centered"
)


## 3. Estilos personalizados con CSS

Streamlit permite inyectar CSS usando `st.markdown` con `unsafe_allow_html=True`.  
Aquí se definen tres clases:

- `.big-title`: un título grande, centrado, con color violeta claro.
- `.sub-title`: subtítulo centrado, color gris claro.
- `.card`: estilo de “tarjeta” con borde redondeado, fondo semitransparente y tipografía un poco más pequeña.

Esto se hace para que la interfaz se vea más limpia y agradable, especialmente en modo oscuro.


In [ ]:
st.markdown("""
<style>
.big-title {
    font-size: 32px;
    text-align: center;
    font-weight: bold;
    color: #b8a8ff;
    margin-bottom: 0.3rem;
}
.sub-title {
    text-align: center;
    color: #dddddd;
    margin-bottom: 1.5rem;
}
.card {
    padding: 0.9rem 1rem;
    border-radius: 0.8rem;
    border: 1px solid #6e6e8a;
    background-color: rgba(255,255,255,0.08);
    color: #ffffff;
    font-size: 0.95rem;
    line-height: 1.35rem;
}
</style>
""", unsafe_allow_html=True)


## 4. Cabecera e introducción de la herramienta

En esta sección se muestra el título principal y una breve descripción usando las clases CSS definidas.

Luego se construye una fila de tres columnas (`st.columns(3)`) con tarjetas que resumen:

1. **Objetivo**: seguimiento de recuperación motora post-ictus.
2. **Entrada**: qué tipo de archivo CSV se espera (ventana de 10 s de EMG combinada).
3. **Salida**: gesto predicho y distribución de probabilidades.

Esto sirve como “pantalla de bienvenida” para que el usuario entienda qué hace la app antes de interactuar.


In [ ]:
# Título y subtítulo
st.markdown('<p class="big-title">BioGest EMG – Seguimiento motor post-ictus</p>', unsafe_allow_html=True)
st.markdown(
    '<p class="sub-title">'
    'Esta herramienta permite cargar ventanas de señal EMG de antebrazo y clasificarlas en tres gestos: '
    '<b>extensión</b>, <b>grip</b> y <b>desviación ulnar</b>, usando el modelo entrenado en Edge Impulse.'
    '</p>',
    unsafe_allow_html=True
)

# Tarjetas informativas en 3 columnas
with st.container():
    col1, col2, col3 = st.columns(3)
    with col1:
        st.markdown(
            '<div class="card"><b>Objetivo</b><br/>'
            'Apoyar el seguimiento de la recuperación motora en pacientes post-ictus, '
            'automatizando la clasificación de gestos de mano en base a EMG.</div>',
            unsafe_allow_html=True
        )
    with col2:
        st.markdown(
            '<div class="card"><b>Entrada</b><br/>'
            'Ventana de 10 s de EMG multicanal combinada en una sola señal (RMS / magnitud) en formato CSV.</div>',
            unsafe_allow_html=True
        )
    with col3:
        st.markdown(
            '<div class="card"><b>Salida</b><br/>'
            'Gesto predicho y distribución de probabilidades para cada clase.</div>',
            unsafe_allow_html=True
        )

st.markdown("---")

st.markdown("### 🧪 Demo interactiva")
st.write(
    "Sigue estos pasos: "
    "**1)** carga una ventana EMG en formato CSV, "
    "**2)** visualiza la señal y "
    "**3)** clasifica el gesto usando la red neuronal entrenada."
)


## 5. Carga del modelo de Edge Impulse

Aquí se define la ruta del modelo `.eim` y se crea un `ImpulseRunner`, que es el objeto encargado de ejecutar inferencias.

- `runner = ImpulseRunner(MODEL_PATH)`: crea una instancia del runner usando el archivo del modelo.
- `runner.init()`: inicializa el modelo y devuelve `model_info`, un diccionario con datos del modelo (etiquetas, cantidad de características, etc.).
- `LABELS`: lista de nombres de las clases del modelo.
- `INPUT_LEN`: cantidad de muestras que el modelo espera como entrada (longitud del vector de características).

Estos parámetros se usan luego para ajustar la señal EMG al tamaño correcto antes de clasificar.


In [ ]:
# Cargar modelo EIM
MODEL_PATH = "model/emg-hand-gestures-linux-x86_64-v5.eim"

runner = ImpulseRunner(MODEL_PATH)
model_info = runner.init()

LABELS = model_info["model_parameters"]["labels"]
INPUT_LEN = model_info["model_parameters"]["input_features_count"]


## 6. Función de predicción `predict_eim`

Esta función encapsula la lógica de preparación de la señal y de llamada al modelo:

1. **Ajuste de longitud**  
   - Si la señal es más larga que `INPUT_LEN`, se recorta.
   - Si es más corta, se rellena con ceros al final (padding).

2. **Conversión de tipo**  
   - Se asegura que la señal esté en `float32` y en un vector plano (`flatten()`).

3. **Clasificación**  
   - Se llama a `runner.classify(signal)`.
   - `scores` contiene un diccionario con `{etiqueta: probabilidad}`.
   - `pred_label` es la clase con mayor probabilidad.
   - `probs` es una lista de probabilidades en el mismo orden que `LABELS`.

La función devuelve el nombre de la clase predicha y la lista de probabilidades, que luego se usan en la interfaz.


In [ ]:
def predict_eim(signal: np.ndarray):

    # Ajustar longitud requerida
    if len(signal) > INPUT_LEN:
        signal = signal[:INPUT_LEN]
    elif len(signal) < INPUT_LEN:
        padding = np.zeros(INPUT_LEN - len(signal))
        signal = np.concatenate([signal, padding])

    signal = np.array(signal, dtype=np.float32).flatten()

    # Importante: usar solo el vector
    res = runner.classify(signal)

    scores = res["result"]["classification"]
    pred_label = max(scores, key=scores.get)
    probs = [scores[lbl] for lbl in LABELS]

    return pred_label, probs


## 7. Paso 1: subir archivo CSV con la ventana EMG

Se crea un título de sección y un componente de subida de archivo:

- `st.file_uploader(...)` muestra un botón que permite seleccionar un archivo `.csv`.
- Si el usuario no sube nada, `uploaded_file` será `None`.
- Solo cuando se detecta un archivo se ejecuta el bloque condicional `if uploaded_file is not None:`.

Dentro del bloque:

1. Se lee el CSV con `pd.read_csv(uploaded_file)`.
2. Se muestra un “preview” con `st.dataframe(df.head())`.
3. Se extrae la **última columna** del DataFrame y se intenta convertir a `float`.  
   - Si falla (por ejemplo, porque hay texto), se muestra un error con `st.error` y se detiene la app con `st.stop()`.

La idea es que la última columna del CSV sea la señal EMG ya combinada (por ejemplo, RMS de varios canales).


In [ ]:
st.markdown("### 1️⃣ Cargar ventana EMG (CSV)")
uploaded_file = st.file_uploader(
    "Sube un archivo CSV con tu ventana de EMG combinada.",
    type=["csv"]
)

if uploaded_file is not None:
    df = pd.read_csv(uploaded_file)

    st.write("Vista previa del archivo:")
    st.dataframe(df.head())

    try:
        signal = df.iloc[:, -1].astype(float).values
    except Exception:
        st.error("⚠️ La última columna no contiene valores numéricos. Revisa tu archivo CSV.")
        st.stop()


## 8. Paso 2: graficar la señal EMG

Si el archivo se cargó correctamente, se pasa al siguiente bloque dentro del mismo `if`:

- Se crea un título de sección: “Visualizar señal EMG”.
- Se genera una figura de `matplotlib` y se dibuja la señal (`ax.plot(signal, ...)`).
- Se configuran título y ejes.
- Se muestra el gráfico con `st.pyplot(fig)`.

Esto le permite al usuario verificar visualmente que la señal cargada tiene sentido antes de enviarla al modelo.


In [ ]:
    # Paso 2: graficar señal
    st.markdown("### 2️⃣ Visualizar señal EMG")

    fig, ax = plt.subplots(figsize=(8, 3))
    ax.plot(signal, color="#b8a8ff")
    ax.set_title("Ventana EMG cargada")
    ax.set_xlabel("Muestra")
    ax.set_ylabel("Magnitud")
    st.pyplot(fig)


## 9. Paso 3: clasificar el gesto con el modelo

En este último paso interactivo:

1. Se muestra el título “Clasificar gesto”.
2. Se coloca un mensaje indicando que hay que presionar el botón.
3. `st.button("🔮 Clasificar gesto")` crea un botón.  
   - Cuando el usuario lo presiona, ese botón devuelve `True` en esa ejecución, activando el bloque `if`.

Dentro del bloque del botón:

- Se llama a `predict_eim(signal)` usando la señal que se cargó anteriormente.
- Se muestra un mensaje de éxito con el gesto predicho, en mayúsculas.
- Se construye un DataFrame con las probabilidades por clase.
- Se muestra un gráfico de barras (`st.bar_chart(prob_df)`) que representa la distribución de probabilidades de cada gesto.

Si el usuario no ha subido archivo, se muestra un `st.info` indicando que debe subir un CSV para habilitar los pasos 2 y 3.


In [ ]:
    # Paso 3: clasificar gesto
    st.markdown("### 3️⃣ Clasificar gesto")

    st.write("Presiona el botón para ejecutar el modelo sobre esta ventana:")

    if st.button("🔮 Clasificar gesto"):
        pred, probs = predict_eim(signal)

        st.success(f"### Gesto predicho: **{pred.upper()}**")

        prob_df = pd.DataFrame({
            "Clase": LABELS,
            "Probabilidad": probs
        }).set_index("Clase")

        st.subheader("Distribución de probabilidades")
        st.bar_chart(prob_df)

else:
    st.info("⬆️ Sube un archivo CSV para habilitar los pasos 2 y 3.")


## 10. Resumen del flujo interno de la app

En términos de lógica, el frontend funciona así:

1. **Inicialización**  
   - Se configura la página y el estilo.
   - Se carga el modelo de Edge Impulse y se obtienen parámetros (etiquetas y longitud de entrada).

2. **Interfaz estática**  
   - Se muestra la cabecera, descripción y tarjetas de objetivo/entrada/salida.
   - Se describe el flujo de uso (pasos 1, 2 y 3).

3. **Interacción con el usuario**  
   - El usuario sube un CSV → se lee y se extrae la señal de la última columna.
   - Si la señal no es numérica, se detiene y se muestra un error.
   - Si todo está bien, se grafica la señal y se habilita el botón de clasificación.

4. **Clasificación**  
   - Al presionar el botón, se normaliza la longitud de la señal.
   - Se envía la señal al modelo con `runner.classify`.
   - Se obtiene la clase con mayor probabilidad y se muestran:
     - El gesto predicho.
     - El gráfico de barras con las probabilidades.

Este diseño mantiene el frontend simple para el usuario final: solo necesita preocuparse por preparar bien el CSV y seguir los pasos guiados en la interfaz.
